In [ ]:
mutable struct Population 
    std::vector<Organism*> organisms;

    std::vector<Species*> species;  // Species in the Population. Note that the species should comprise all the genomes 

    // ******* Member variables used during reproduction *******
    std::vector<Innovation*> innovations;  // For holding the genetic innovations of the newest generation
    int cur_node_id;  //Current label number available
    double cur_innov_num;

    int last_species;  //The highest species number

    // ******* Fitness Statistics *******
    double mean_fitness;
    double variance;
    double standard_deviation;

    int winnergen; //An integer that when above zero tells when the first winner appeared

    // ******* When do we need to delta code? *******
    double highest_fitness;  //Stagnation detector
    int highest_last_changed; //If too high, leads to delta coding

    // Separate the Organisms into species
    bool speciate();

    // Print Population to a file specified by a string 
    bool print_to_file(std::ostream& outFile);

    // Print Population to a file in speciated order with comments separating each species
    bool print_to_file_by_species(std::ostream& outFile);
    bool print_to_file_by_species(char *filename);

    // Prints the champions of each species to files starting with directory_prefix
    // The file name are as follows: [prefix]g[generation_num]cs[species_num]
    // Thus, they can be indexed by generation or species
    bool print_species_champs_tofiles(char *directory_prefix,int generation);

    // Run verify on all Genomes in this Population (Debugging)
    bool verify();

    // Turnover the population to a new generation using fitness 
    // The generation argument is the next generation
    bool epoch(int generation);

    // *** Real-time methods *** 

    // Places the organisms in species in order from best to worst fitness 
    bool rank_within_species();

    // Construct off of a single spawning Genome 
    Population(Genome *g,int size);

    // Construct off of a single spawning Genome without mutation
    Population(Genome *g,int size, float power);

    //MSC Addition
    // Construct off of a vector of genomes with a mutation rate of "power"
    Population(std::vector<Genome*> genomeList, float power);

    bool clone(Genome *g,int size, float power);

    //// Special constructor to create a population of random topologies     
    //// uses Genome(int i, int o, int n,int nmax, bool r, double linkprob) 
    //// See the Genome constructor for the argument specifications
    //Population(int size,int i,int o, int nmax, bool r, double linkprob);

    // Construct off of a file of Genomes 
    Population(const char *filename);

    // It can delete a Population in two ways:
    //    -delete by killing off the species
    //    -delete by killing off the organisms themselves (if not speciated)
    // It does the latter if it sees the species list is empty
    ~Population();



};

}

In [ ]:
Population::Population(Genome *g,int size) {
winnergen=0;
highest_fitness=0.0;
highest_last_changed=0;
spawn(g,size);
}

Population::Population(Genome *g,int size, float power) {
winnergen=0;
highest_fitness=0.0;
highest_last_changed=0;
clone(g, size, power);
}

//Population::Population(int size,int i,int o, int nmax, bool r, double linkprob) {    
//int count;
//Genome *new_genome; 

//cout<<"Making a random pop"<<endl;

//winnergen=0;
//highest_fitness=0.0;
//highest_last_changed=0;

//for(count=0;count<size;count++) {
//new_genome=new Genome(count,i,o,randint(0,nmax),nmax,r,linkprob);
//organisms.push_back(new Organism(0,new_genome,1));
//}

//cur_node_id=i+o+nmax+1;;
//cur_innov_num=(i+o+nmax)*(i+o+nmax)+1;

//cout<<"Calling speciate"<<endl;
//speciate(); 

//}


//MSC Addition
//Added the ability for a population to be spawned
//off of a vector of Genomes.  Useful when converging.
Population::Population(std::vector<Genome*> genomeList, float power) {

winnergen=0;
highest_fitness=0.0;
highest_last_changed=0;

int count;
Genome *new_genome;
Organism *new_organism;

//Create size copies of the Genome
//Start with perturbed linkweights
for (std::vector<Genome*>::iterator iter = genomeList.begin(); iter != genomeList.end(); ++iter)
{

    new_genome=(*iter); 
    if(power>0)
        new_genome->mutate_link_weights(power,1.0,GAUSSIAN);
    //new_genome->mutate_link_weights(1.0,1.0,COLDGAUSSIAN);
    new_genome->randomize_traits();
    new_organism=new Organism(0.0,new_genome,1);
    organisms.push_back(new_organism);
}

//Keep a record of the innovation and node number we are on
cur_node_id=new_genome->get_last_node_id();
cur_innov_num=new_genome->get_last_gene_innovnum();

//Separate the new Population into species
speciate();
}

Population::Population(const char *filename) {

char curword[128];  //max word size of 128 characters
char curline[1024]; //max line size of 1024 characters
char delimiters[] = " \n";

Genome *new_genome;

winnergen=0;

highest_fitness=0.0;
highest_last_changed=0;

cur_node_id=0;
cur_innov_num=0.0;

int curwordnum = 0;

std::ifstream iFile(filename);
if (!iFile) {
    printf("Can't open genomes file for input");
    return;
}

else {
    bool md = false;
    char metadata[128];
    //Loop until file is finished, parsing each line
    while (!iFile.eof()) 
    {
        iFile.getline(curline, sizeof(curline));
        std::stringstream ss(curline);
        //strcpy(curword, NEAT::getUnit(curline, 0, delimiters));
        ss >> curword;
        //std::cout << curline << std::endl;

        //Check for next
        if (strcmp(curword,"genomestart")==0) 
        {
            //strcpy(curword, NEAT::getUnit(curline, 1, delimiters));
            //int idcheck = atoi(curword);

            int idcheck;
            ss >> idcheck;

            // If there isn't metadata, set metadata to ""
            if(md == false)  {
                strcpy(metadata, "");
            }
            md = false;

            new_genome=new Genome(idcheck,iFile);
            organisms.push_back(new Organism(0,new_genome,1, metadata));
            if (cur_node_id<(new_genome->get_last_node_id()))
                cur_node_id=new_genome->get_last_node_id();

            if (cur_innov_num<(new_genome->get_last_gene_innovnum()))
                cur_innov_num=new_genome->get_last_gene_innovnum();
        }
        else if (strcmp(curword,"/*")==0) 
        {
            // New metadata possibly, so clear out the metadata
            strcpy(metadata, "");
            curwordnum=1;
            //strcpy(curword, NEAT::getUnit(curline, curwordnum++, delimiters));
            ss >> curword;

            while(strcmp(curword,"*/")!=0)
            {
                // If we've started to form the metadata, put a space in the front
                if(md) {
                    strncat(metadata, " ", 128 - strlen(metadata));
                }

                // Append the next word to the metadata, and say that there is metadata
                strncat(metadata, curword, 128 - strlen(metadata));
                md = true;

                //strcpy(curword, NEAT::getUnit(curline, curwordnum++, delimiters));
                ss >> curword;
            }
        }
        //Ignore comments - they get printed to screen
        //else if (strcmp(curword,"/*")==0) {
        //	iFile>>curword;
        //	while (strcmp(curword,"*/")!=0) {
        //cout<<curword<<" ";
        //		iFile>>curword;
        //	}
        //	cout<<endl;

        //}
        //Ignore comments surrounded by - they get printed to screen
    }

    iFile.close();

    speciate();

}
}


Population::~Population() {

std::vector<Species*>::iterator curspec;
std::vector<Organism*>::iterator curorg;
//std::vector<Generation_viz*>::iterator cursnap;

if (species.begin()!=species.end()) {
    for(curspec=species.begin();curspec!=species.end();++curspec) {
        delete (*curspec);
    }
}else {
    for(curorg=organisms.begin();curorg!=organisms.end();++curorg) {
        delete (*curorg);
    }
}

for (std::vector<Innovation*>::iterator iter = innovations.begin(); iter != innovations.end(); ++iter)
    delete *iter;

#Delete the snapshots
#    for(cursnap=generation_snapshots.begin();cursnap!=generation_snapshots.end();++cursnap) {
#        delete (*cursnap);
#    }
}